In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the custom stock data
ftnt = pd.read_csv('FTNT.csv', parse_dates=['Date'], index_col='Date')
vrtx = pd.read_csv('VRTX.csv', parse_dates=['Date'], index_col='Date')
nvda = pd.read_csv('NVDA.csv', parse_dates=['Date'], index_col='Date')
wm = pd.read_csv('WM.csv', parse_dates=['Date'], index_col='Date')
ecl = pd.read_csv('ECL.csv', parse_dates=['Date'], index_col='Date')

# Ensure index consistency and convert to daily frequency if necessary
ftnt = ftnt.asfreq('D')
vrtx = vrtx.asfreq('D')
nvda = nvda.asfreq('D')
wm = wm.asfreq('D')
ecl = ecl.asfreq('D')

# Read and clean the whale_returns and algo_returns data
whale = pd.read_csv('whale_returns.csv', parse_dates=['Date'], index_col='Date').asfreq('D')
algo = pd.read_csv('algo_returns.csv', parse_dates=['Date'], index_col='Date').asfreq('D')

# Read S&P 500 Index data and S&P 60 TSX ETF data
sp500 = pd.read_csv('SP500_INDEXSP_INX.csv', parse_dates=['Date'], index_col='Date').asfreq('D')
sp60 = pd.read_csv('SP60_TSE_XIU.csv', parse_dates=['Date'], index_col='Date').asfreq('D')

# Remove '$' sign and convert to float
sp500['Close'] = sp500['Close'].str.replace('$', '').astype(float)

# Calculate daily returns for each stock and indices
daily_returns_stocks = pd.concat([ftnt['Close'].pct_change(), vrtx['Close'].pct_change(),
                                   nvda['Close'].pct_change(), wm['Close'].pct_change(),
                                   ecl['Close'].pct_change()], axis=1)

# Calculate the equally weighted portfolio returns
weights = np.array([1/5] * 5)
equally_weighted_portfolio = daily_returns_stocks.dot(weights)

# Create a DataFrame for the equally weighted portfolio
ewp_df = pd.DataFrame(equally_weighted_portfolio, columns=['Equally Weighted Portfolio'])

# Combine all returns data including S&P 500 and S&P 60 TSX ETF daily returns
combined_returns = pd.concat([ewp_df, sp500['Close'].pct_change().rename("S&P 500 Returns"),
                              sp60['Close'].pct_change().rename("S&P 60 TSX ETF Returns"),
                              whale, algo], axis=1)

# Calculate cumulative returns for all data
cumulative_returns = (1 + combined_returns).cumprod() - 1

# Plot cumulative returns
plt.figure(figsize=(15, 10))
for column in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[column], label=column)
plt.title('Cumulative Returns of All Portfolios Including Equally Weighted Portfolio')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend(loc="upper left")
plt.show()

# Calculate and plot the correlation matrix for combined returns
correlation_matrix = combined_returns.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.05)
plt.title('Correlation Matrix of Combined Portfolio Returns')
plt.show()

# Export combined returns to CSV
combined_returns.to_csv('combined_portfolio_returns.csv')
print("Combined portfolio returns exported to 'combined_portfolio_returns.csv'.")


AttributeError: Can only use .str accessor with string values!